In [ ]:
import sys
import os.path as op

basename = "/Users/Adrian/Desktop/WDDS/Tweet-Classification-Diabetes-Distress/"
path_utils = op.join(basename , "utils")
sys.path.insert(0, path_utils)

from sys_utils import load_library
from tweet_utils import tweet_vectorizer

from preprocess import Preprocess
prep = Preprocess()

In [ ]:

def check_emoticonEmoji_in_tweet(tweet):
    
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.tokenize(tweet)
    
    for word in tweet: 
        match_emoticon = Patterns.EMOTICONS_PATTERN.findall(word)
        
        # if emoticon found
        if match_emoticon:
            if match_emoticon[0] is not ":":
                print(tweet)
                print(matc_emoticon[0])
                print()
                return True
            
        # if emoji found    
        if word in UNICODE_EMOJI: 

            emot_cat = EMOJI_TO_CATEGORY[UNICODE_EMOJI[word]]
            if emot_cat != "": return True
    
            
        
    return False


def check_emotKeywords(tweet):
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = tweet.replace("nerve pain", "")
    tweet = tweet.replace("nerve damage", "")
    tweet = tweet.replace("pain of neuropathy", "")
    tweet = prep.tokenize(tweet)
    tweet = [word for word in tweet if word not in ["healthcare"]]

    
    for word in tweet:
        for emot in Emotions.emotions_full_list:
            
            if emot == word: 
                print(tweet)
                print(emot)
                print()
                return True
            
    return False


tweet["emotion"] = tweet.text.map(lambda tweet: check_emoticon_in_tweet(tweet) or check_emotKeywords(tweet))




In [ ]:
import operator

emotico_dict = {}

for tweet in tweet.text:
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.tokenize(tweet)
    for word in tweet: 
        match_emoticon = Patterns.EMOTICONS_PATTERN.findall(word)
        
        # if emoticon found
        if match_emoticon:
            if match_emoticon[0] in emotico_dict: emotico_dict[match_emoticon[0]] += 1
            else : emotico_dict[match_emoticon[0]] = 1
                
        # if emoji found    
        if word in UNICODE_EMOJI: 
            emot_cat = EMOJI_TO_CATEGORY[UNICODE_EMOJI[word]]
            if emot_cat != "": 
                if word in emotico_dict: emotico_dict[word] += 1
                else : emotico_dict[word] = 1
                
sorted_x = sorted(emotico_dict.items(), key=operator.itemgetter(1))
print(sorted_x)

In [ ]:
emojo_dict = {}

for tweet in tweet.text:
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.tokenize(tweet)
    
    for word in tweet:
        for emot in Emotions.emotions_full_list:
            
            if emot == word: 
                if word in emojo_dict: emojo_dict[emot] += 1
                else : emojo_dict[emot] = 1
                    
sorted_y = sorted(emojo_dict.items(), key=operator.itemgetter(1))
print(sorted_y)